In [1]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from composio_langchain import ComposioToolSet, App
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain_core.tools import ToolException
from typing import List
import warnings

warnings.filterwarnings("ignore")

/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = ChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhace_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to."),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhace_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool.func),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [3]:
composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
agent_tools = composio_toolset.get_tools(apps=[App.NOTION])

In [5]:
agent = MyAgent(tools=agent_tools)

In [6]:
agent.myinvoke("Crie uma nova tarefa 'Testar composio' para o Rafael Magalhaes, com data de 27/07")



> Entering new AgentExecutor chain...


python(77222) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Invoking: `notion_search_notion_page` with `{'query': 'tarefas'}`


{'execution_details': {'executed': True}, 'search_results': {'object': 'list', 'results': [], 'next_cursor': None, 'has_more': False, 'type': 'page_or_database', 'page_or_database': {}, 'request_id': '38e24a45-270f-48d5-848b-078ba5f0de40'}}
Invoking: `notion_search_notion_page` with `{'query': 'tasks'}`


{'execution_details': {'executed': True}, 'search_results': {'object': 'list', 'results': [{'object': 'database', 'id': '7ac1a262-d8e3-49d6-9ff4-f882ac0164fc', 'cover': None, 'icon': None, 'created_time': '2024-07-25T13:46:00.000Z', 'created_by': {'object': 'user', 'id': '7232a348-787e-4e5b-9d96-9f6b7a453f52'}, 'last_edited_by': {'object': 'user', 'id': '2d7748cf-543d-4f57-b6be-3bb1aefc2109'}, 'last_edited_time': '2024-07-26T20:08:00.000Z', 'title': [{'type': 'text', 'text': {'content': 'Tasks', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, '

In [7]:
agent.myinvoke("Em vez de colocar o nome dele na coluna 'Person', coloque na coluna 'Comentários'")



> Entering new AgentExecutor chain...

Invoking: `notion_search_notion_page` with `{'query': 'Tarefas', 'filter_value': 'database'}`


{'execution_details': {'executed': True}, 'search_results': {'object': 'list', 'results': [], 'next_cursor': None, 'has_more': False, 'type': 'page_or_database', 'page_or_database': {}, 'request_id': '74e2fca0-5226-4889-9984-3c46dfabfb7e'}}
Invoking: `notion_search_notion_page` with `{'filter_value': 'database'}`
responded: Não consegui encontrar um banco de dados para "Tarefas". Vou listar todos os bancos de dados disponíveis para verificar se há algum que possa ser usado.

{'execution_details': {'executed': True}, 'search_results': {'object': 'list', 'results': [{'object': 'database', 'id': '7ac1a262-d8e3-49d6-9ff4-f882ac0164fc', 'cover': None, 'icon': None, 'created_time': '2024-07-25T13:46:00.000Z', 'created_by': {'object': 'user', 'id': '7232a348-787e-4e5b-9d96-9f6b7a453f52'}, 'last_edited_by': {'object': 'user', 'id': '2d7748cf-543d-4f57-b6be-3b

In [9]:
from langchain_community.document_loaders import AsyncHtmlLoader

notion_url = "https://morossini.notion.site/NOTION-TASKS-AGENT-9119d3eb96794320b78f6d41155fc74f"
loader = AsyncHtmlLoader(notion_url)
docs = loader.load()
print(docs)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.86it/s]


[Document(metadata={'source': 'https://morossini.notion.site/NOTION-TASKS-AGENT-9119d3eb96794320b78f6d41155fc74f', 'title': 'Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.', 'description': "A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team", 'language': 'No language found.'}, page_content='<!doctype html><html class="notion-html"><head lang="en"><meta charset="utf-8"><meta name="viewport" content="width=device-width,height=device-height,initial-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover"><title>Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.</title><meta name="description" content="A new tool that blends your everyday work apps into one. It\'s the all-in-one workspace for you and your team"><noscript><meta http-equiv="refresh" content="0;url=https://www.notion.so/disabled-javascript.html"></noscript><script>var ua=window.navigator.userAgent,isEdgeHT

In [10]:
type(docs)

list

In [11]:
docs[0]

Document(metadata={'source': 'https://morossini.notion.site/NOTION-TASKS-AGENT-9119d3eb96794320b78f6d41155fc74f', 'title': 'Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.', 'description': "A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team", 'language': 'No language found.'}, page_content='<!doctype html><html class="notion-html"><head lang="en"><meta charset="utf-8"><meta name="viewport" content="width=device-width,height=device-height,initial-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover"><title>Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.</title><meta name="description" content="A new tool that blends your everyday work apps into one. It\'s the all-in-one workspace for you and your team"><noscript><meta http-equiv="refresh" content="0;url=https://www.notion.so/disabled-javascript.html"></noscript><script>var ua=window.navigator.userAgent,isEdgeHTM

In [13]:
dir(docs[0])

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enfo

In [14]:
docs[0].page_content

'<!doctype html><html class="notion-html"><head lang="en"><meta charset="utf-8"><meta name="viewport" content="width=device-width,height=device-height,initial-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover"><title>Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.</title><meta name="description" content="A new tool that blends your everyday work apps into one. It\'s the all-in-one workspace for you and your team"><noscript><meta http-equiv="refresh" content="0;url=https://www.notion.so/disabled-javascript.html"></noscript><script>var ua=window.navigator.userAgent,isEdgeHTML=/Windows.*Edge/i.test(ua),isChrome=!isEdgeHTML&&/Chrome/i.test(ua),isSafari=!isEdgeHTML&&!isChrome&&/Safari/i.test(ua),isFirefox=/Firefox/i.test(ua),isIE=/Trident/i.test(ua);function isUnsupportedBrowser(){var e;if(isChrome){if((e=ua.match(/Chrome\\/(\\d+)/i))&&e[1])return parseInt(e[1],10)<=84}else if(isSafari){if((e=ua.match(/Version\\/(\\d+)/i))&&e[1])return parseInt(e[1],

In [21]:
from pprint import pprint
pprint(docs[0].dict)

<bound method BaseModel.dict of Document(metadata={'source': 'https://morossini.notion.site/NOTION-TASKS-AGENT-9119d3eb96794320b78f6d41155fc74f', 'title': 'Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.', 'description': "A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team", 'language': 'No language found.'}, page_content='<!doctype html><html class="notion-html"><head lang="en"><meta charset="utf-8"><meta name="viewport" content="width=device-width,height=device-height,initial-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover"><title>Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.</title><meta name="description" content="A new tool that blends your everyday work apps into one. It\'s the all-in-one workspace for you and your team"><noscript><meta http-equiv="refresh" content="0;url=https://www.notion.so/disabled-javascript.html"></noscript><script>var ua=wind